In [1]:
import os

from pathlib import Path

import tensorflow as tf

import pandas as pd 
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras import losses

In [2]:
print(tf.__version__)

2.13.1


In [3]:
def _get_imdb_data_from_web():
    _LOCAL_FNAME = "./imdb_full_dataset.csv"
    local_file = Path(_LOCAL_FNAME)
    
    if local_file.is_file():
        return pd.read_csv(_LOCAL_FNAME)
    
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

    dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                        untar=True, cache_dir='.',
                                        cache_subdir='')

    dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
    
    _, _, pos_files = next(os.walk("./aclImdb/train/pos/"))
    _, _, neg_files = next(os.walk("./aclImdb/train/neg/"))
    
    pos_corpus = []
    for fname in pos_files:
        with open("./aclImdb/train/pos/" + fname) as f:
            text = f.read()
            pos_corpus.append(text)

    pos_labels = [1] * len(pos_corpus)
    
    neg_corpus = []
    for fname in neg_files:
        with open("./aclImdb/train/neg/" + fname) as f:
            text = f.read()
            neg_corpus.append(text)

    neg_labels = [0] * len(neg_corpus)
    
    df = pd.DataFrame({"text": pos_corpus + neg_corpus, "label": pos_labels + neg_labels})
    
    df = df.sample(frac=1.0, random_state=0)
    
    df.to_csv(_LOCAL_FNAME)
    
    return df 

In [4]:
df = _get_imdb_data_from_web()

In [5]:
df.head()

,Unnamed: 0,text,label
0,14149,I had two reasons for watching this swashbuckl...,0
1,8946,"This is, in my opinion, a very good film, espe...",1
2,22378,I knew this film was supposed to be so bad it ...,0
3,12162,"When the US entered World War I, the governmen...",1
4,4879,Few movies can be viewed almost 60 years later...,1


In [6]:
print("class one average", np.mean(df["label"]))

class one average 0.5


In [7]:
def preprocess_text(input_text: str) -> str:
    s = input_text.lower()
    s = s.replace('<br />', ' ')
    return s
    # return token_pattern.findall(s)

In [8]:
df["text"] = df["text"].map(preprocess_text)

In [9]:
tok = tf.keras.preprocessing.text.Tokenizer()

In [10]:
from sklearn.model_selection import train_test_split


In [11]:
text_train, text_test, y_train, y_test = train_test_split(
    df["text"], 
    df["label"], 
    test_size=0.2,
    random_state=1,
)

In [12]:
tok.fit_on_texts(text_train)

In [13]:
len(tok.word_index)


80476

In [14]:
X_train = tok.texts_to_sequences(text_train)
X_test = tok.texts_to_sequences(text_test)

X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=512, padding="post")
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=512, padding="post")

In [15]:
model = tf.keras.Sequential([
    layers.Embedding(input_dim=len(tok.index_word) + 1, output_dim=16, input_length=512),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(1, activation="sigmoid")]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 16)           1287632   
                                                                 
 dropout (Dropout)           (None, 512, 16)           0         
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1287649 (4.91 MB)
Trainable params: 1287649 (4.91 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [16]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.5))


In [17]:
epochs = 20
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs)

Epoch 1/20
625/625 [==============================] - 86s 135ms/step - loss: 0.6786 - binary_accuracy: 0.6575 - val_loss: 0.6523 - val_binary_accuracy: 0.7720
Epoch 2/20
625/625 [==============================] - 40s 63ms/step - loss: 0.6030 - binary_accuracy: 0.7884 - val_loss: 0.5612 - val_binary_accuracy: 0.8056
Epoch 3/20
625/625 [==============================] - 21s 34ms/step - loss: 0.5047 - binary_accuracy: 0.8339 - val_loss: 0.4828 - val_binary_accuracy: 0.8256
Epoch 4/20
625/625 [==============================] - 17s 27ms/step - loss: 0.4271 - binary_accuracy: 0.8648 - val_loss: 0.4246 - val_binary_accuracy: 0.8488
Epoch 5/20
625/625 [==============================] - 13s 20ms/step - loss: 0.3688 - binary_accuracy: 0.8860 - val_loss: 0.3848 - val_binary_accuracy: 0.8574
Epoch 6/20
625/625 [==============================] - 10s 16ms/step - loss: 0.3256 - binary_accuracy: 0.8989 - val_loss: 0.3564 - val_binary_accuracy: 0.8648
Epoch 7/20
625/625 [==============================]

In [18]:
X_train = tok.texts_to_sequences(text_train)
X_test = tok.texts_to_sequences(text_test)

X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=128, padding="post")
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=128, padding="post")

In [19]:
model = tf.keras.Sequential([
    layers.Embedding(input_dim=len(tok.index_word) + 1, output_dim=64, input_length=128),
    layers.Dropout(0.2),
    layers.LSTM(64, return_sequences=False),
    layers.Dropout(0.2),
    layers.Dense(1, activation="sigmoid")]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 128, 64)           5150528   
                                                                 
 dropout_2 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5183617 (19.77 MB)
Trainable params: 5183617 (19.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.5))


In [21]:
epochs = 20
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs)

Epoch 1/20
625/625 [==============================] - 55s 85ms/step - loss: 0.4344 - binary_accuracy: 0.7897 - val_loss: 0.3281 - val_binary_accuracy: 0.8568
Epoch 2/20
625/625 [==============================] - 13s 21ms/step - loss: 0.2124 - binary_accuracy: 0.9250 - val_loss: 0.3341 - val_binary_accuracy: 0.8660
Epoch 3/20
625/625 [==============================] - 9s 14ms/step - loss: 0.1158 - binary_accuracy: 0.9645 - val_loss: 0.3863 - val_binary_accuracy: 0.8562
Epoch 4/20
625/625 [==============================] - 9s 15ms/step - loss: 0.0761 - binary_accuracy: 0.9776 - val_loss: 0.5091 - val_binary_accuracy: 0.8512
Epoch 5/20
625/625 [==============================] - 7s 11ms/step - loss: 0.0579 - binary_accuracy: 0.9836 - val_loss: 0.5694 - val_binary_accuracy: 0.8430
Epoch 6/20
625/625 [==============================] - 6s 10ms/step - loss: 0.0530 - binary_accuracy: 0.9858 - val_loss: 0.5987 - val_binary_accuracy: 0.8432
Epoch 7/20
625/625 [==============================] - 5s